In [ ]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install accelerate
!pip install datasets

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-lsqzpw5b
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-lsqzpw5b
  Resolved https://github.com/huggingface/transformers.git to commit e65502951593a76844e872fee9c56b805598538a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.43.0.dev0-py3-none-any.whl size=9337412 sha256=e6fee39523003c38b167998bf9c627348b069772322f1597921ca35cb0c13781
  Stored in directory: /tmp/pip-ephem-wheel-cache-nfsah47z/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import Dataset
import torch
from transformers import pipeline, TrainingArguments, Trainer, DataCollatorForLanguageModeling

# Fine-tuning

### Preparing data for tuning

In [ ]:
# ins = ds['train']['Context']
# inp = ds['train']['Question']
# out = ds['train']['Answer']

# one_row = {
#     'instruction': ins,
#     'input': inp,
#     'output': out
# }
# import pandas as pd
# dataFrame = pd.DataFrame(one_row)
ds = load_dataset("armanc/ScienceQA")

Repo card metadata block was not found. Setting CardData to empty.


In [ ]:
import pandas as pd
data = pd.read_csv('./train.csv')
data.rename(columns={'instruction': 'system', 'input': 'user', 'output': 'assistant'}, inplace=True)
data.columns

Index(['Unnamed: 0', 'system', 'user', 'assistant'], dtype='object')

In [ ]:
print(f"[SYSTEM]: {data['system'][10]}, [USER]: {data['user'][10]}, [ASSISTANT]: {data['assistant'][10]}")

[SYSTEM]: This paper deals with the generalization ability of classifiers trained from non-iid evolutionary-related data in which all training and testing examples correspond to leaves of a phylogenetic tree. For the realizable case, we prove PAC-type upper and lower bounds based on symmetries and matchings in such trees., [USER]: what and testing examples correspond to leaves of a phylogenetic tree ?, [ASSISTANT]: non iid evolutionary related data in which all training and testing examples correspond to leaves of a phylogenetic tree


In [ ]:
def format_llama(rows):

  system = rows['instruction'].strip()
  user = rows['input'].strip()
  assistant = rows['output'].strip()

  converted = f"<|system|>{system}</s>\n"
  converted += f"<|user|>{user}</s>\n"
  converted += f"<|assistant|>{assistant}</s>"

  return converted


converted = []

In [ ]:
import csv

with open('./train.csv', 'r', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        converted.append(format_llama(row))

with open('./train.txt', 'w', encoding='utf-8') as f:
    for item in converted:
        f.write(item + '\n\n')  # Add two newlines between examples
print("DONE!")

DONE!


### Preparing Model for Fine tuning

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

torch.cuda.empty_cache()

model_name = "TinyLlama/TinyLlama_v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

In [ ]:
# Prepare the dataset
def load_dataset(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return [line.strip() for line in f if line.strip()]


train_texts = load_dataset('./train.txt')
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)

train_dataset = Dataset.from_dict({"text": train_texts})
tokenized_train = train_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/154375 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
)

In [ ]:
# # Start training
# trainer.train()

In [ ]:
# Save the model
model.save_pretrained("./finetuned_llama_model")
tokenizer.save_pretrained("./finetuned_llama_model")

# Inferencing

In [ ]:
pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")
# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
question = "social media has become a popular medium for what ?"

messages = [
    {
        "role": "system",
        "content": "In recent times, social media has become a popular medium for many election campaigns. It not only allows candidates to reach out to a large section of the electorate, it is also a potent medium for people to express their opinion on the proposed policies and promises of candidates. Analyzing social media data is challenging as the text can be noisy, sparse and even multilingual. In addition, the information may not be completely trustworthy, particularly in the presence of propaganda, promotions and rumors. In this paper we describe our work for analyzing election campaigns using social media data. Using data from the 2012 US presidential elections and the 2013 Philippines General elections, we provide detailed experiments on our methods that use granger causality to identify topics that were most “causal” for public opinion and which in turn, give an interpretable insight into “elections topics” that were most important. Our system was deployed by the largest media organization in the Philippines during the 2013 General elections and using our work, the media house able to identify and report news stories much faster than competitors and reported higher TRP ratings during the election.",
    },
    {"role": "user", "content": f"{question}"},
]

In [ ]:
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)

In [ ]:
print(outputs[0]["generated_text"])

<|system|>
In recent times, social media has become a popular medium for many election campaigns. It not only allows candidates to reach out to a large section of the electorate, it is also a potent medium for people to express their opinion on the proposed policies and promises of candidates. Analyzing social media data is challenging as the text can be noisy, sparse and even multilingual. In addition, the information may not be completely trustworthy, particularly in the presence of propaganda, promotions and rumors. In this paper we describe our work for analyzing election campaigns using social media data. Using data from the 2012 US presidential elections and the 2013 Philippines General elections, we provide detailed experiments on our methods that use granger causality to identify topics that were most “causal” for public opinion and which in turn, give an interpretable insight into “elections topics” that were most important. Our system was deployed by the largest media organiz

# Testing the model

In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [1]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset, load_metric

# Load your model and tokenizer
model_name = "TinyLlama/TinyLlama_v1.1"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [2]:
# Create a text generation pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [3]:
# Load a dataset for evaluation (e.g., from the Hugging Face datasets library)
dataset = load_dataset("squad", split="validation[:1%]")

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [4]:
# Define a function to generate responses and evaluate them
def evaluate_model(model, dataset):
    results = []
    for example in dataset:
        prompt = example["context"][:100]  # Take the first 100 characters of context as a prompt
        generated = generator(prompt, max_length=200, num_return_sequences=1)
        response = generated[0]["generated_text"]
        results.append((prompt, response))

    return results

In [5]:
# Run the evaluation
results = evaluate_model(model, dataset)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


KeyboardInterrupt: 

In [ ]:
# Print a few examples
for prompt, response in results[:5]:
    print(f"Prompt: {prompt}")
    print(f"Response: {response}")
    print("-" * 50)

In [ ]:
# Example: Calculating Perplexity (assuming you have a language modeling task)
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
import torch

def perplexity(model, tokenizer, text):
    encodings = tokenizer(text, return_tensors='pt')
    input_ids = encodings.input_ids.to(model.device)

    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
        loss = outputs.loss

    perplexity = torch.exp(loss)
    return perplexity.item()

sample_text = "The quick brown fox jumps over the lazy dog."
perplexity = perplexity(model, tokenizer, sample_text)
print(f"Perplexity: {perplexity}")

